In [3]:
import numpy as np
import pandas as pd
import requests
import math

In [2]:
stoxx = pd.read_csv('sp_500_stocks.csv')
stoxx

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [5]:
from secrets import IEX_CLOUD_API_TOKEN

ImportError: cannot import name 'IEX_CLOUD_API_TOKEN' from 'secrets' (c:\Users\obafemi\AppData\Local\Programs\Python\Python310\lib\secrets.py)

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

NameError: name 'IEX_CLOUD_API_TOKEN' is not defined

In [ ]:
price = data['latestPrice']
market_cap = data['marketCap']/1000000000000
market_cap

In [ ]:
for stox in stoxx['Ticker']:
    print(stox)

In [ ]:
column_info = ['Ticker', 'Stock Price', 'Market Cap', 'Number of shares to buy']
info_dataframe = pd.DataFrame(columns = column_info)
info_dataframe

In [ ]:
info_dataframe.append(pd.Series([
    symbol,
    price,
    market_cap,
    'N/A'
],index=column_info),ignore_index=True)

In [ ]:
column_info = ['Ticker', 'Stock Price', 'Market Cap', 'Number of shares to buy']
info_dataframe = pd.DataFrame(columns = column_info)
for stox in stoxx['Ticker'][:1]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stox}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    info_dataframe=info_dataframe.append(pd.Series([
        stox,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],index=column_info), ignore_index=True)

In [ ]:
info_dataframe

In [ ]:
stock_chunks = []
for i in range(0, len(stoxx['Ticker']), 100):
    chunk = stoxx['Ticker'][i:i + 100]
    stock_chunks.append(chunk)

In [ ]:
stock_chunks

In [ ]:
stock_chunks
stock_string = []
for i in range (0, len(stock_chunks)):
    stock_string.append(','.join(stock_chunks[i]))
    
info_dataframe = pd.DataFrame(columns = column_info)
for string in stock_string:
    batch_api_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call).json()
    for sym in string.split(','):
        try:
            info_dataframe = info_dataframe.append(pd.Series([
                sym,
                data[sym]['quote']['latestPrice'],
                data[sym]['quote']['marketCap'],
                'N/A'

            ], index=column_info), ignore_index=True)
        except KeyError:
            info_dataframe = info_dataframe.append(pd.Series([
                sym,
                'N/A',
                'N/A',
                'N/A'

            ], index=column_info), ignore_index=True)
        

In [ ]:
info_dataframe.head(137)

In [ ]:
stock_string

In [ ]:
while True:
    portfolio_size = input("What is your portfolio size?")
    if portfolio_size.isdigit():
        portfolio = float(portfolio_size)
        break
    else:
        print('Enter a number instead')
#         portfolio_size = input("What is your portfolio size?")

In [ ]:
portfolio

In [ ]:
position_size = portfolio/len(info_dataframe.index)
position_size
apple_shares = position_size/500
print(math.floor(apple_shares))

In [ ]:
info_dataframe

In [ ]:
for i in range (0, len(info_dataframe.index)):
    try:
        info_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size/int(info_dataframe.loc[i, 'Stock Price']))
    except ValueError:
        info_dataframe.loc[i, 'Number of shares to buy'] = 'N/A'
info_dataframe

In [ ]:
len(info_dataframe.index)

In [ ]:
info_dataframe.to_csv('Recommended trades')